### Future

A Future represents an eventual result of an asynchronous operation.

- It is an awaitable. Coro can await on it (A) until they have a result, exception set, or cancelled. It can be awaited multiple times with same results.
- Typical purpose is to enable low-level callback-based code to work with high-level async/await code, by allowing customised conditions for waiting and resuming.
- Key attributes:
    - State are one of: pending, cancelled, done
    - Result is set when state is transitioned to done (B)
- Task is a subclass of Future (P), callbacks list is implemented in Future (Q)

In [1]:
import asyncio
import datetime
import time

In [4]:
issubclass(asyncio.Task, asyncio.Future)  # P

True

In [7]:
[i for i in dir(asyncio.Future) if "callback" in i]  # Q

['_callbacks', 'add_done_callback', 'remove_done_callback']

### Future example

- Use future to control when the sleep task is done (A)
- Use a task that monitors time and call set_result (B)
- Cede control to event loop via an awaitable (C)
- The `_sleep_watcher` coro 's task will be invoked once per full cycle of the event loop
    - On each resumption, it'll check time, then pause and cede control back to event loop again (D)

In [ ]:
class YieldToEventLoop:
    def __await__(self):
        yield  # C


async def _sleep_watcher(future, time_to_wake):
    while True:
        if time.time() >= time_to_wake:  # E
            # This marks the future as done.
            future.set_result(None)  # B
            break
        else:
            await YieldToEventLoop()  # D

In [ ]:
async def async_sleep(seconds: float):
    future = asyncio.Future()
    time_to_wake = time.time() + seconds
    # Add the watcher-task to the event loop.
    watcher_task = asyncio.create_task(_sleep_watcher(future, time_to_wake))
    # Block until the future is marked as done.
    await future  # A

In [11]:
async def other_work():
    print("I like work. Work work.")


async def main():
    # Add a few other tasks to the event loop, so there's something
    # to do while asynchronously sleeping.
    work_tasks = [
        asyncio.create_task(other_work()),
        asyncio.create_task(other_work()),
        asyncio.create_task(other_work()),
    ]
    print(
        "Beginning asynchronous sleep at time: "
        f"{datetime.datetime.now().strftime("%H:%M:%S")}."
    )
    await asyncio.create_task(async_sleep(3))
    print(
        "Done asynchronous sleep at time: "
        f"{datetime.datetime.now().strftime("%H:%M:%S")}."
    )
    # asyncio.gather effectively awaits each task in the collection.
    await asyncio.gather(*work_tasks)


await main()

Beginning asynchronous sleep at time: 17:51:40.
I like work. Work work.
I like work. Work work.
I like work. Work work.
Done asynchronous sleep at time: 17:51:43.
